In [ ]:
!pip install --upgrade transformers huggingface_hub
!pip install peft==0.4.0
!pip install bitsandbytes==0.41.3
!pip install datasets
!pip install wandb
!pip install huggingface_hub
!pip install accelerate==0.26.0
!pip install trl==0.4.7

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset

from transformers import(
AutoTokenizer,
AutoModelForCausalLM,
TrainingArguments,
Trainer
)

from peft import LoraConfig, get_peft_model, TaskType
import bitsandbytes as bnb
import os
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
dataset = load_dataset("Dahoas/code-review-instruct-critique-revision-python")
dataset = dataset['train'] #.select(range(10))
shuffled_dataset = dataset.shuffle(seed = 42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2, seed = 42)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['body', 'answer', 'comments', 'meta_data', 'question_id', 'prompt', 'response'],
        num_rows: 7569
    })
    test: Dataset({
        features: ['body', 'answer', 'comments', 'meta_data', 'question_id', 'prompt', 'response'],
        num_rows: 1893
    })
})

In [ ]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Verify the sizes of the splits
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

Train size: 7569
Test size: 1893


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast = False)
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

# Alternatively, add a custom padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize embeddings if needed

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def format_example(example):
    """
    Formats the dataset into a consistent structure using the given prompt and response.
    """
    prompt = example.get('prompt', '').strip()
    response = example.get('response', '').strip()

    formatted_string = f"""
    ### Task:
    Analyze the provided problem and optimize the given Python code. Include constructive feedback and an improved version of the code.

    ### Prompt:
    {prompt}

    ### Instructions:
    1. Read and understand the original code and its problem statement.
    2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
    3. Rewrite the code to address the critique.

    ### Response:
    {response}
    """

    return formatted_string


In [ ]:
print(format_example(split_dataset["train"][0]))


    ### Task:
    Analyze the provided problem and optimize the given Python code. Include constructive feedback and an improved version of the code.

    ### Prompt:
    Question: <p>I wrote this script to add some config to a Juniper device.  It works well but the whole user input validation section (yes or no section) seems a little messy.</p>

<p>Is there a better way I could have done it?</p>

<pre><code>from jnpr.junos import Device
from jnpr.junos.utils.config import Config
import getpass

#Grab credentials
username = raw_input("Enter your username:")
password = getpass.getpass("Enter your password:")

#Confirmation of commands
commands = open("commands" , "r")
commands_to_commit = commands.read()
print ""
print "The commands you are about to commit are: "
print commands_to_commit
print ""
print "Do you want to continue to run these commands?"
confirm = raw_input("Y or N: ")
    if confirm in ['n','no','N','NO','No']:
            print "Exiting..."
            quit()
while conf

In [ ]:
def tokenize_function(examples):
    """
    Tokenize the formatted examples from the dataset for the given structure.
    """
    # Create formatted strings for the batch
    formatted_examples = [
        format_example({
            'prompt': prompt,
            'response': response,
        })
        for prompt, response in zip(
            examples['prompt'],
            examples['response'],
        )
    ]


    # Tokenize the formatted examples
    tokenized = tokenizer(
        formatted_examples,
        padding="max_length",
        truncation=True,
        max_length=1024,
    )

    # Set the labels
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized


In [ ]:
# Define the final zero-shot prompt
zero_shot_prompt = """
### Task:
Analyze the provided problem and optimize the given Python code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
<p>I'd like suggestions for optimizing this brute force solution to <a href="http://projecteuler.net/index.php?section=problems&amp;id=1">problem 1</a>. The algorithm currently checks every integer between 3 and 1000. I'd like to cut as many unnecessary calls to <code>isMultiple</code> as possible:</p>
<pre><code>'''
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.
'''
end = 1000

def Solution01():
    ''' Solved by brute force #OPTIMIZE '''
    sum = 0
    for i in range(3, end):
        if isMultiple(i):
            sum += i
    print(sum)

def isMultiple(i):
    return (i % 3 == 0) or (i % 5 == 0)
</code></pre>

### Instructions:
1. Read and understand the original code and its problem statement.
2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
3. Rewrite the code to address the critique.
4. Begin your critique with "Critique:" and your revised code with "Revised Code:".

##Response:

"""


In [ ]:
final_prompt = zero_shot_prompt
input_ids = tokenizer(final_prompt, return_tensors="pt").input_ids
output = model.generate( input_ids, max_length=1024, temperature=0.7, top_p=0.95, repetition_penalty=1.2, num_return_sequences=1, )

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


### Task:
Analyze the provided problem and optimize the given Python code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
<p>I'd like suggestions for optimizing this brute force solution to <a href="http://projecteuler.net/index.php?section=problems&amp;id=1">problem 1</a>. The algorithm currently checks every integer between 3 and 1000. I'd like to cut as many unnecessary calls to <code>isMultiple</code> as possible:</p>
<pre><code>'''
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.
'''
end = 1000

def Solution01():
    ''' Solved by brute force #OPTIMIZE '''
    sum = 0
    for i in range(3, end):
        if isMultiple(i):
            sum += i
    print(sum)

def isMultiple(i):
    return (i % 3 == 0) or (i % 5 == 0)
</code></pre>

### Instructions:
1. Read and understand the original code and its problem 

In [ ]:
few_shot_prompt = """
### Task:
Analyze the provided problem and optimize the given code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
<p>I'd like suggestions for optimizing this brute force solution to <a href="http://projecteuler.net/index.php?section=problems&amp;id=1">problem 1</a>. The algorithm currently checks every integer between 3 and 1000. I'd like to cut as many unnecessary calls to <code>isMultiple</code> as possible:</p>
<pre><code>'''
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.
'''
end = 1000

def Solution01():
    ''' Solved by brute force #OPTIMIZE '''
    sum = 0
    for i in range(3, end):
        if isMultiple(i):
            sum += i
    print(sum)

def isMultiple(i):
    return (i % 3 == 0) or (i % 5 == 0)
</code></pre>

### Instructions:
1. Read and understand the original code and its problem statement.
2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
3. Rewrite the code to address the critique.
4. Begin your critique with "Critique:" and your revised code with "Revised Code:".

### Response:

### Critique:
The algorithm currently checks every integer between 3 and 1000, making unnecessary calls to `isMultiple`, which is inefficient. Instead, we can iterate over multiples of 3 and 5 directly, avoiding repeated checks.

### Revised Code:
```python
end = 1000
total = 0
for i in range(3, end, 3):
    total += i
for i in range(5, end, 5):
    if i % 3 != 0:
        total += i
print(total)"""

new_question = """
### Task:
Analyze the provided problem and optimize the given code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
I have written a function that calculates the factorial of a number recursively. However, the code is not optimized for larger inputs as it can cause a stack overflow. I would like suggestions on optimizing this code:

function factorial(n) {
    if (n === 0 || n === 1) {
        return 1;
    }
    return n * factorial(n - 1);
}

console.log(factorial(5));

### Instructions:
1. Read and understand the original code and its problem statement.
2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
3. Rewrite the code to address the critique.
4. Begin your critique with "Critique:" and your revised code with "Revised Code:".

### Response:
"""

In [ ]:
final_prompt = few_shot_prompt + new_question
input_ids = tokenizer(final_prompt, return_tensors="pt").input_ids
output = model.generate( input_ids, max_length=1024, temperature=0.7, top_p=0.95, repetition_penalty=1.2, num_return_sequences=1, )

In [ ]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


### Task:
Analyze the provided problem and optimize the given code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
<p>I'd like suggestions for optimizing this brute force solution to <a href="http://projecteuler.net/index.php?section=problems&amp;id=1">problem 1</a>. The algorithm currently checks every integer between 3 and 1000. I'd like to cut as many unnecessary calls to <code>isMultiple</code> as possible:</p>
<pre><code>'''
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.
'''
end = 1000

def Solution01():
    ''' Solved by brute force #OPTIMIZE '''
    sum = 0
    for i in range(3, end):
        if isMultiple(i):
            sum += i
    print(sum)

def isMultiple(i):
    return (i % 3 == 0) or (i % 5 == 0)
</code></pre>

### Instructions:
1. Read and understand the original code and its problem stateme

In [ ]:
# Specify columns to retain
columns_to_keep = ['prompt', 'response']

# Remove unnecessary columns dynamically
columns_to_remove_train = [col for col in train_dataset.column_names if col not in columns_to_keep]
columns_to_remove_test = [col for col in test_dataset.column_names if col not in columns_to_keep]

# Tokenize train dataset
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_train,
)

# Tokenize test dataset
test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_test,
)

# Verify tokenized datasets
print("Tokenized Train Dataset Sample:", train_dataset[0])
print("Tokenized Test Dataset Sample:", test_dataset[0])

Tokenized Train Dataset Sample: {'prompt': 'Question: <p>I wrote this script to add some config to a Juniper device.  It works well but the whole user input validation section (yes or no section) seems a little messy.</p>\n\n<p>Is there a better way I could have done it?</p>\n\n<pre><code>from jnpr.junos import Device\nfrom jnpr.junos.utils.config import Config\nimport getpass\n\n#Grab credentials\nusername = raw_input("Enter your username:")\npassword = getpass.getpass("Enter your password:")\n\n#Confirmation of commands\ncommands = open("commands" , "r")\ncommands_to_commit = commands.read()\nprint ""\nprint "The commands you are about to commit are: "\nprint commands_to_commit\nprint ""\nprint "Do you want to continue to run these commands?"\nconfirm = raw_input("Y or N: ")\n    if confirm in [\'n\',\'no\',\'N\',\'NO\',\'No\']:\n            print "Exiting..."\n            quit()\nwhile confirm not in [\'y\',\'Y\',\'yes\',"Yes",\'YES\',\'n\',\'N\',\'NO\',\'No\',\'no\']:\n    print "I

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,  # Rank of the update matrices
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Target specific modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,  # Increase if memory allows
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="wandb",  # Disable reporting to external tools like WandB
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: deepakuday23. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
50,0.923100,0.934524
100,0.930200,0.923848
150,0.906000,0.918745
200,0.908700,0.914725
250,0.926200,0.911887
300,0.878600,0.909861
350,0.901000,0.907660
400,0.872100,0.905228
450,0.877700,0.903675
500,0.891100,0.902968


TrainOutput(global_step=708, training_loss=0.902474259252602, metrics={'train_runtime': 9622.7245, 'train_samples_per_second': 2.36, 'train_steps_per_second': 0.074, 'total_flos': 9.244028078621983e+17, 'train_loss': 0.902474259252602, 'epoch': 2.993261989694808})

In [ ]:
# Save the LoRA adapters
model.save_pretrained("/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1")

('/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1/tokenizer_config.json',
 '/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1/special_tokens_map.json',
 '/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1/tokenizer.model',
 '/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1/added_tokens.json')

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load the base model with 4-bit quantization
# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Load the LoRA adapters
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/Models_LLM_Project/mistral-7b-lora-it-code-optimize-v1")

# Set to evaluation mode
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
         

In [ ]:
def generate_response(prompt):
    """
    Generates a response from the model for a given prompt.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.2,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text[len(prompt):].strip()
    return response

In [ ]:
# Example prompt
# Define the final zero-shot prompt
prompt = """
### Task:
Analyze the provided problem and optimize the given Python code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
# I need suggestions for optimizing this recursive solution to generate Fibonacci numbers.
# The current implementation is inefficient for large numbers. I'd like to improve its performance:

'''
Generate the nth Fibonacci number.
The Fibonacci sequence is defined as:
F(n) = F(n-1) + F(n-2), where F(0) = 0 and F(1) = 1
'''

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Test the function
print(fibonacci(10))

### Instructions:
1. Read and understand the original code and its problem statement.
2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
3. Rewrite the code to address the critique.
4. Begin your critique with "Critique:" and your revised code with "Revised Code:".

##Response:


"""

# Generate response
response = generate_response(prompt)

# Print the response
print("Generated Response:\n", response)


Generated Response:
 ```python
Critique:
This program has two major issues that can be addressed by using recursion instead of iteration. Firstly, it uses too many nested loops which makes the program slow when dealing with larger values of `n`. Secondly, it does not use any data structures such as lists or arrays which would allow us to store multiple values at once rather than having to calculate them one after another each time we call the function.

To fix these problems we will rewrite our algorithm so that it only calls itself twice (once per loop). We also introduce some new variables into play - `i` represents how far through our list/array we are currently looking while `j` keeps track of what value should come next based on previous ones already seen beforehand; finally there exists yet another variable called `k`, which stores whatever result comes out from combining both `i`'s & `j`'s together!

Revised Code:
def fibonaccci_recursive(n):
	if n == 0:
		return 0
	elif n < 0:


In [ ]:
# Example prompt
# Define the final zero-shot prompt
prompt = """
### Task:
Analyze the provided problem and optimize the given Python code. Provide constructive feedback and generate an improved version of the code.

### Prompt:
# I need suggestions for optimizing this recursive solution to generate Fibonacci numbers.
# The current implementation is inefficient for large numbers. I'd like to improve its performance:

'''
Generate the nth Fibonacci number.
The Fibonacci sequence is defined as:
F(n) = F(n-1) + F(n-2), where F(0) = 0 and F(1) = 1
'''

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Test the function
print(fibonacci(10))

### Instructions:
1. Read and understand the original code and its problem statement.
2. Provide a critique of the code, highlighting inefficiencies and areas for improvement.
3. Rewrite the code to address the critique.
4. Begin your critique with "Critique:" and your revised code with "Revised Code:".

##Response:


"""

# Generate response
response = generate_response(prompt)

# Print the response
print("Generated Response:\n", response)


Generated Response:
 <pre><code>class Solution:
    def __init__(self):
        self._cache = {}
    
    @staticmethod
    def fibonacci_number(n):
        """Returns the nth Fibonacci number."""
        
        # Check cache first
        try:
            return Solution()._cache[n]
            
        except KeyError:
            pass
                     
        # If not found then calculate it recursively
        elif n &gt;= 1:
            return (Solution()).fibonacci_number((n - 1)) \
                + (Solution()).fibonacci_number((n - 2))
</code></pre>

<p>This class has two methods.</p>

<ol>
<li>A static method <code>fibonacci_number()</code>, which calculates the next Fibonacci number based on the previous ones stored in the cache.</li>
<li>A private instance method <code>__init__()</code>. This initializes the cache dictionary by setting default values for all keys that don’t exist yet.</li>
</ol>

<hr>

<blockquote>
<pre><code>&gt;&gt;&gt; Solution().fibonacci_number(